# Stable Diffusion + Telegram bot
[Github Repo](https://github.com/blackestwhite/tele-sd)

## Donations
Ethereum: `0xE76fc1CE4d3ffFEA12A5686618844408C327357b`

## steps
- create a telegram bot
- set the token on the code
- run all


## How to use
go to your bot on telegram send the Prompts separated by `__`, (two underscores).
before the separator will be the positive prompt, after that will be the negative prompt

## Mount Google Drive

In [ ]:
import os
from google.colab import drive

drivePath = '/content/drive'

if os.path.isdir(drivePath) == False:
    drive.mount(drivePath)

## Install Requirements

In [ ]:
%pip install --quiet nest-asyncio transformers diffusers python-telegram-bot accelerate triton scipy ftfy spacy xformers

## Import and Setup Bot + SD

In [ ]:
from telegram import Update, Bot
from telegram.ext import ApplicationBuilder, CommandHandler, ContextTypes, MessageHandler, filters
import nest_asyncio
from diffusers import DiffusionPipeline
import torch
import asyncio
import random

base = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
)
base.to("cuda")
refiner = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    text_encoder_2=base.text_encoder_2,
    vae=base.vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
)
refiner.to("cuda")

nest_asyncio.apply()
token = "Your Telegram Bot Token Here"

## Setup Handler for bot

In [ ]:
default_steps = 40
high_noise_frac = 0.8

async def generate(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
  text = update.message.text
  text = text.replace("\n", "")
  prompt = ''
  neg = ''
  if '__' in text:
    prompt = text.split("__")[0]
    neg = text.split("__")[1]
  else:
    prompt = text
  seed = random.randint(0, 2147483647)
  image = base(
    prompt=prompt,
    negative_prompt=neg,
    num_inference_steps=default_steps,
    denoising_end=high_noise_frac,
    output_type="latent",
    generator = torch.Generator("cuda").manual_seed(seed)
  ).images
  image = refiner(
      prompt=prompt,
      num_inference_steps=default_steps,
      denoising_start=high_noise_frac,
      image=image,
  ).images[0]
  image.save("./image.png")
  await update.message.reply_photo(photo="./image.png")

app = ApplicationBuilder().token(token).build()

app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, generate))

app.run_polling()